In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

dataset_mlt = pd.read_csv('/content/drive/MyDrive/Maltese_Unimorph_database_final.txt', sep="  ", header=None, error_bad_lines=False)

dataset_copy_mlt=dataset_mlt.copy()
list_data_mlt = dataset_copy_mlt.values.tolist()
print(list_data_mlt[0:10])


In [ ]:
split_data_mlt=[]

def split(corpus, spit_data):
  for sets in corpus:
    for x in sets:
      parts = x.split("\t")
      spit_data.append(parts)

split(list_data_mlt,split_data_mlt)
print(split_data_mlt[0:100])

print(len(split_data_mlt))

In [ ]:
# removing duplicates
def remove_duplicates(data_array):
    unique_data_array = []
    second_element_set = set()
    for subarray in data_array:
        if subarray[1] not in second_element_set:
            unique_data_array.append(subarray)
            second_element_set.add(subarray[1])
    return unique_data_array

unique_data_mlt = remove_duplicates(split_data_mlt)
print(unique_data_mlt[0:100])

#getting the longest word for padding 
len_longest=0
for string in unique_data_mlt:
    x=string[1]
    if len(x)>len_longest:
      len_longest=len(x)

In [ ]:
from collections import Counter
import pandas as pd

# splitting into words and grammar
def split_word_and_grammar(unique_data_array):
    last_elements = [subarray[-1] for subarray in unique_data_array]
    word_list = [(subarray[0],subarray[1]) for subarray in unique_data_array]
    return word_list, last_elements

word_list_mlt, last_elements_mlt = split_word_and_grammar(unique_data_mlt)
print(last_elements_mlt[0:100])
'''
# converting to char level
word_list_char_mlt = [[char for char in word] for word in word_list_mlt]
word_list_char_ita = [[char for char in word] for word in word_list_ita]
'''

# splitting grammar
def splitting_grammar(last_elements):
    list_of_lists = []
    for info in last_elements:
        parts = info.split(';')
        num_parts = [int(part) if part.isdigit() else part for part in parts]
        list_of_lists.append(num_parts)
    return list_of_lists

list_of_lists_mlt = splitting_grammar(last_elements_mlt)


# adding to lists in alt order
split_data = []
list_of_lists = []
for i in range(len(word_list_mlt)):
      # adding mlt word and grammar
      array=(word_list_mlt[i][0],) + word_list_mlt[i][1:]
      split_data.append(array)
      list_of_lists.append(list_of_lists_mlt[i])

print(split_data[0:100])
print(list_of_lists[0:100])



In [ ]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import torch
import torch.cuda

In [ ]:
# splitting data
tokeniser = AutoTokenizer.from_pretrained("MLRS/BERTu",char_level=True) #,char_level=True
model = AutoModelForMaskedLM.from_pretrained("MLRS/BERTu")

#tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-cased",char_level=True)
#model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")

from sklearn.model_selection import train_test_split
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

X = split_data
y = list_of_lists

X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.2)
print(X_train[0:100])
print(y_train[0:100])
print(len(X_train))

In [ ]:
import sklearn
from sklearn import preprocessing
from collections import defaultdict

# tokenize the training set
token_indexes = tokeniser(X_train, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_train_x = token_indexes['input_ids'].to(device).squeeze()
mask_train_x = token_indexes['attention_mask'].to(device).squeeze()

# tokenize the test set
new_tuples = [(("",) + t[1:]) for t in X_test]
token_indexes_test = tokeniser(new_tuples, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_test_x = token_indexes_test['input_ids'].to(device).squeeze()
mask_test_x = token_indexes_test['attention_mask'].to(device).squeeze()

# padding w/0 so gramm info is all the same length
def padding_grammar(Y):
    max_len = max(len(info) for info in Y)
    padded_array = [info + ['<PAD>'] * (max_len - len(info)) for info in Y]
    return padded_array
padded_morph_info_train = padding_grammar(y_train)
padded_morph_info_test = padding_grammar(y_test)
print(padded_morph_info_train[0:10])

all_morph_info = padded_morph_info_train + padded_morph_info_test
# creating a dictionary for mapping 
morph_vocab = defaultdict(lambda: len(morph_vocab)) 

for i in range(len(all_morph_info[0])):
    # extracting the unique values at this position
    values = set(info[i] for info in all_morph_info)
    # assignign index
    for value in values:
        morph_vocab[value]

print(morph_vocab)


In [ ]:
import matplotlib.pyplot as plt

all_inflections = []
for key in morph_vocab:
    all_inflections.append(key)
print(all_inflections)

# count the occurrences of each inflection in the list of lists
inflection_counts = []
for inflection in all_inflections:
    count = 0
    for inflection_list in y_train:
        if inflection in inflection_list:
            count += 1
    for inflection_list in y_test:
        if inflection in inflection_list:
            count += 1
    inflection_counts.append(count)

print(inflection_counts)

# calculate the frequencies of each inflection
total_inflections = max(inflection_counts)
inflection_freqs = [count for count in inflection_counts]
print(total_inflections)

# create a list of integers representing the positions of the bars
x_values = range(len(all_inflections))

# plot the counts as a bar chart
fig, ax = plt.subplots()
ax.bar(x_values, inflection_freqs)

# set axis labels and title
ax.set_xlabel('Inflections', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.set_title('Inflection Counts', fontsize=14)

# set the x-axis tick labels to the inflection names
ax.set_xticks(x_values)
ax.set_xticklabels(all_inflections, rotation=90, fontsize=8)

# set the y-axis ticks
max_count = max(inflection_counts)
tick_step = max_count // 10 if max_count > 10 else 1
ax.set_yticks(range(0, max_count + tick_step, tick_step))

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(8)
    
# show plot
plt.show()
'''
import numpy as np

# create a 2D array to store the confusion matrix
confusion_matrix = np.zeros((len(list_of_lists), len(list_of_lists)))

# loop over all pairs of lists and count the number of shared elements
for i in range(len(list_of_lists)):
    for j in range(i+1, len(list_of_lists)):
        count = len(set(list_of_lists[i]).intersection(set(list_of_lists[j])))
        confusion_matrix[i][j] = count
        confusion_matrix[j][i] = count

# print the confusion matrix
print(confusion_matrix)
'''

In [ ]:
from tabulate import tabulate

# create a ranking list of the inflections and the corresponding number of inflections
ranked_inflections = sorted(zip(all_inflections, inflection_counts), key=lambda x: x[1], reverse=True)

# create a list of lists containing the ranking, inflection, and count data
table_data = []
for i, (inflection, count) in enumerate(ranked_inflections):
    table_data.append([i+1, inflection, count])

# print the table
print(tabulate(table_data, headers=["Rank", "Inflection", "Count"]))



In [ ]:
import numpy as np

# create a 2D array to store the confusion matrix
confusion_matrix = np.zeros((len(list_of_lists), len(list_of_lists)))

# loop over all pairs of lists and count the number of shared elements
for i in range(len(list_of_lists)):
    for j in range(i+1, len(list_of_lists)):
        count = len(set(list_of_lists[i]).intersection(set(list_of_lists[j])))
        confusion_matrix[i][j] = count
        confusion_matrix[j][i] = count

# print the confusion matrix
print(confusion_matrix)
